In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://practicum-content.s3.us-west-1.amazonaws.com/data-analyst-eng/moved_chicago_weather_2017.html"

response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, "html.parser")
weather_table = soup.find("table", attrs={"id": "weather_records"})

dates = []
temperatures = []
descriptions = []

rows = weather_table.find_all("tr")

for row in rows[1:]:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]
    dates.append(cols[0])
    temperatures.append(cols[1])
    descriptions.append(cols[2])


weather_records = pd.DataFrame(
    {"Date and time": dates, "Temperature": temperatures, "Description": descriptions}
)

print(weather_records)

           Date and time Temperature       Description
0    2017-11-01 00:00:00     276.150     broken clouds
1    2017-11-01 01:00:00     275.700  scattered clouds
2    2017-11-01 02:00:00     275.610   overcast clouds
3    2017-11-01 03:00:00     275.350     broken clouds
4    2017-11-01 04:00:00     275.240     broken clouds
..                   ...         ...               ...
692  2017-11-29 20:00:00     281.340        few clouds
693  2017-11-29 21:00:00     281.690      sky is clear
694  2017-11-29 22:00:00     281.070        few clouds
695  2017-11-29 23:00:00     280.060      sky is clear
696  2017-11-30 00:00:00     278.460      sky is clear

[697 rows x 3 columns]


## Trabajar con bases de datos

**Ejercicio 1**

Imprime el campo company_name. Encuentra la cantidad de viajes en taxi para cada compañía de taxis para el 15 y 16 de noviembre de 2017, asigna al campo resultante el nombre trips_amount e imprímelo también. Ordena los resultados por el campo trips_amount en orden descendente.

In [ ]:
SELECT c.company_name, COUNT(t.trip_id) AS trips_amount
FROM cabs c
JOIN trips t ON c.cab_id = t.cab_id
WHERE DATE(t.start_ts) IN ('2017-11-15', '2017-11-16')
GROUP BY c.company_name
ORDER BY trips_amount DESC;

**Ejercicio 2**

Encuentra la cantidad de viajes para cada empresa de taxis cuyo nombre contenga las palabras "Yellow" o "Blue" del 1 al 7 de noviembre de 2017. Nombra la variable resultante trips_amount. Agrupa los resultados por el campo company_name.

In [ ]:
SELECT c.company_name, COUNT(t.trip_id) AS trips_amount
FROM cabs c
JOIN trips t ON c.cab_id = t.cab_id
WHERE DATE(t.start_ts) BETWEEN '2017-11-01' AND '2017-11-07'
AND (c.company_name LIKE '%Yellow%' OR c.company_name LIKE '%Blue%')
GROUP BY c.company_name;


**Ejercicio 3**

Del 1 al 7 de noviembre de 2017, las empresas de taxis más populares fueron Flash Cab y Taxi Affiliation Services. Encuentra el número de viajes de estas dos empresas y asigna a la variable resultante el nombre trips_amount. Junta los viajes de todas las demás empresas en el grupo "Other". Agrupa los datos por nombres de empresas de taxis. Asigna el nombre company al campo con nombres de empresas de taxis. Ordena el resultado en orden descendente por trips_amount.

In [ ]:
SELECT 
    CASE 
        WHEN c.company_name IN ('Flash Cab', 'Taxi Affiliation Services') THEN c.company_name 
        ELSE 'Other' 
    END AS company,
    COUNT(t.trip_id) AS trips_amount
FROM 
    cabs c
JOIN 
    trips t ON c.cab_id = t.cab_id
WHERE 
    DATE(t.start_ts) BETWEEN '2017-11-01' AND '2017-11-07'
GROUP BY 
    CASE 
        WHEN c.company_name IN ('Flash Cab', 'Taxi Affiliation Services') THEN c.company_name 
        ELSE 'Other' 
    END
ORDER BY 
    trips_amount DESC;


**Ejercicio 4**

Recupera los identificadores de los barrios de O'Hare y Loop de la tabla neighborhoods.

In [ ]:
SELECT neighborhood_id, name
FROM neighborhoods
WHERE name LIKE '%Hare' OR name LIKE 'Loop';

**Ejercicio 5**

Para cada hora recupera los registros de condiciones meteorológicas de la tabla weather_records. Usando el operador CASE, divide todas las horas en dos grupos: Bad si el campo description contiene las palabras rain o storm, y Good para los demás. Nombra el campo resultante weather_conditions. La tabla final debe incluir dos campos: fecha y hora (ts) y weather_conditions.

In [ ]:
SELECT 
    w.ts AS date_and_time,
    CASE 
        WHEN w.description LIKE '%rain%' OR w.description LIKE '%storm%' THEN 'Bad' 
        ELSE 'Good' 
    END AS weather_conditions
FROM 
    weather_records w;


**Ejercicio 6**

Recupera de la tabla de trips todos los viajes que comenzaron en el Loop (pickup_location_id: 50) el sábado y terminaron en O'Hare (dropoff_location_id: 63). Obtén las condiciones climáticas para cada viaje. Utiliza el método que aplicaste en la tarea anterior. Recupera también la duración de cada viaje. Ignora los viajes para los que no hay datos disponibles sobre las condiciones climáticas.

Las columnas de la tabla deben estar en el siguiente orden:

- start_ts
- weather_conditions
- duration_seconds

Ordena por trip_id.

In [ ]:
SELECT
    t.start_ts,
    CASE
        WHEN w.description LIKE '%rain%' OR w.description LIKE '%storm%' THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions,
    t.duration_seconds
FROM
    trips t
INNER JOIN neighborhoods n ON t.pickup_location_id = n.neighborhood_id
INNER JOIN weather_records w ON DATE_TRUNC('hour', t.start_ts) = DATE_TRUNC('hour', w.ts)
WHERE
    n.name = 'Loop'
    AND EXTRACT(DOW FROM t.start_ts) = 6
    AND t.dropoff_location_id = 63
ORDER BY
    t.trip_id;
